In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from copy import deepcopy

In [ ]:
path_to_normal_sensors_folder = r"C:\Users\orson\Desktop\Myself\HCI\UWiSchool\Projects\UWEXP\Data\UWEXPI\UW_I_features"
path_to_sleep_merged_file = r"C:\Users\orson\Desktop\Myself\HCI\UWiSchool\Projects\UWEXP\Data\UWEXPI_SampleData\glue\aggregated_horizontal_sleep.csv"
path_to_ema_merged_file = r"C:\Users\orson\Desktop\Myself\HCI\UWiSchool\Projects\UWEXP\Data\UWEXPI_SampleData\glue\aggregated-horizontal-numVal-internalID.csv"
path_to_baseline_merged_file = r"C:\Users\orson\Desktop\Myself\HCI\UWiSchool\Projects\UWEXP\Data\UWEXPI_SampleData\glue\merged_by_row_uw.csv"
path_to_save_giant_table = r"C:\Users\orson\Desktop\Myself\HCI\UWiSchool\Projects\UWEXP\Data\UWEXPI_SampleData\glue"
pid_list_file = r"C:\Users\orson\Desktop\Myself\HCI\UWiSchool\Projects\UWEXP\Data\UWEXPI_SampleData\glue\pids.txt"

# Get the table for sensors (excluding fitbit)

In [ ]:
with open(pid_list_file, "r") as f:
    pid_list = f.readlines()
pid_list = [x.strip() for x in pid_list]

In [ ]:
pid_list = []
for i in os.listdir(path_to_normal_sensors_folder):
    pid_list.append(i)

In [ ]:
pid_list = ["002", "014"]

In [ ]:
normal_sensors = ["activity_android", "activity_ios", "applications", "audio", "battery", "bluetooth",
    "calls", "locations", "screen", "wifi"]
epochs = ["allday", "morning", "afternoon", "evening", "night"]

In [ ]:
# deprecated
def get_sleep_table(path_to_sleep_folder, pid):
    df_sleep = pd.read_csv("{}/PID{}_sleep.csv".format(path_to_sleep_folder,
                                                 pid),
                           sep=',',
                           encoding = "ISO-8859-1")
    df_sleep["participantID"] = pid
    df_sleep.rename({"participantID":"pid", "date":"time"}, inplace = True, axis = 1)
    sleep_feature_column = [x for x in df_sleep.columns if x not in ["pid","time"]]
    df_sleep_main = copy.deepcopy(df_sleep[df_sleep["isMainSleep"] == True])
    df_sleep_main.drop(["isMainSleep"], inplace = True, axis = 1)
    df_sleep_notmain = copy.deepcopy(df_sleep[df_sleep["isMainSleep"] != True])
    df_sleep_notmain.drop(["isMainSleep"], inplace = True, axis = 1)
    df_sleep_main.rename(dict(zip(sleep_feature_column, [x + "_main" for x in sleep_feature_column])), inplace = True, axis = 1)
    df_sleep_notmain.rename(dict(zip(sleep_feature_column, [x + "_notmain" for x in sleep_feature_column])), inplace = True, axis = 1)
    df_sleep_merge = df_sleep_main.merge(df_sleep_notmain, on = ["pid", "time"], how = "outer")
    return df_sleep_merge

In [ ]:
# pid = pid_list[0]
giant_table_sensor = pd.DataFrame()
giant_table_sensor_pids = []

admin_cols = ["PID", "date",  "epoch", "weekday", "grouping", "epoch_weekday_grouping_abbreviated"]

for pid in pid_list:
    sensors_tables = {}
    for sensor in normal_sensors:
        sensors_tables[sensor] = {}
        df_buf = pd.read_csv("{}/{}/{}.txt".format(path_to_normal_sensors_folder,
                                                                     pid,sensor),
                                                                     sep='\t',
                                                                     encoding = "ISO-8859-1")
        df_buf["pid"] = pid
        df_buf.rename({"time":"date", "pid":"PID"}, axis = 1, inplace = True)
        df_buf["date"] = pd.to_datetime(df_buf["date"])
        # currenlty we only look at daily data
        df_buf = df_buf[(df_buf["grouping"] == "day") & (df_buf["weekday"] == "wk")]
        df_buf.drop(columns = ["weekday",
                                "grouping",
                                "epoch_weekday_grouping_abbreviated"],
                    inplace = True)
#         sensors_tables[sensor]["df"] = deepcopy(df_buf)
        sensors_tables[sensor]["feature_cols"] = [x for x in df_buf.columns if x not in admin_cols]
        sensors_tables[sensor]["epochs"] = {}
#         sensors_tables[sensor]["epoch_merged"] = pd.DataFrame()
        for epoch in epochs:
            sensors_tables[sensor]["epochs"][epoch] = deepcopy(df_buf[df_buf["epoch"] == epoch]).drop(columns = ["epoch"])
            sensors_tables[sensor]["epochs"][epoch].rename(dict(zip(
                sensors_tables[sensor]["feature_cols"],
                [x + "_" + epoch for x in sensors_tables[sensor]["feature_cols"]])), axis = 1, inplace = True)
    giant_table_sensor_pid = pd.DataFrame()
    android_ios_flag = sensors_tables["activity_ios"]["epochs"][epochs[0]].shape[0] == 0
    for sensor in normal_sensors:
        if (android_ios_flag):
            if (sensor == "activity_ios"): continue
        else:
            if (sensor == "activity_android"): continue
        giant_table_sensor_pid_epoch = deepcopy(sensors_tables[sensor]["epochs"][epochs[0]])
        for epoch in epochs[1:]:
            giant_table_sensor_pid_epoch = giant_table_sensor_pid_epoch.merge(sensors_tables[sensor]["epochs"][epoch],
                                                                             on = ["date", "PID"], how = "outer")
        # reorder the table
        cols_reorder = []
        for col in sensors_tables[sensor]["feature_cols"]:
            for epoch in epochs:
                cols_reorder.append(col + "_" + epoch)
        cols_reorder = ["PID", "date"] + cols_reorder 
        assert len(cols_reorder) == len(giant_table_sensor_pid_epoch.columns)
        giant_table_sensor_pid_epoch = giant_table_sensor_pid_epoch[cols_reorder]
        
        # add sensor prefix
        sensor_name = sensor
        if ("activity" in sensor_name):
            sensor_name = "activity"
        giant_table_sensor_pid_epoch.columns = [sensor_name + "_" + x if x not in ["PID","date"] else x for x in giant_table_sensor_pid_epoch.columns]
            
        if (giant_table_sensor_pid.shape[0] == 0):
            giant_table_sensor_pid = deepcopy(giant_table_sensor_pid_epoch)
        else:
            giant_table_sensor_pid = giant_table_sensor_pid.merge(giant_table_sensor_pid_epoch, 
                                                                  on = ["date", "PID"], how = "outer")

#     giant_table_sensor_pid = giant_table_sensor_pid.merge(df_sleep_merge, on = ["PID", "date"], how = "outer")
    print(giant_table_sensor_pid.shape)
    giant_table_sensor_pids.append(deepcopy(giant_table_sensor_pid))

In [ ]:
giant_table_sensor = pd.concat(giant_table_sensor_pids)
# giant_table_sensor.set_index(["PID","date"], inplace = True)

In [ ]:
original_columns = giant_table_sensor_pids[0].columns

In [ ]:
giant_table_sensor = giant_table_sensor[original_columns]

In [ ]:
giant_table_sensor["date"] = pd.to_datetime(giant_table_sensor["date"])

In [ ]:
giant_table_sensor.reset_index(drop = True, inplace = True)

In [ ]:
empty_rate = giant_table_sensor.isna().sum(axis = 0) / giant_table_sensor.shape[0]

In [ ]:
giant_table_sensor = giant_table_sensor.drop(empty_rate[empty_rate == 1].index,axis = 1)

In [ ]:
giant_table_sensor.head()

# Get the table for sleep 

In [ ]:
giant_table_sleep = pd.read_csv(path_to_sleep_merged_file)
giant_table_sleep["PID"] = giant_table_sleep["PID"].apply(lambda x : "pid{0:0>3}".format(str(x)))
giant_table_sleep["date"] = pd.to_datetime(giant_table_sleep["date"])
# giant_table_sleep.set_index(["PID","date"], inplace=True)

# Get the table for EMA

In [ ]:
giant_table_EMA = pd.read_csv(path_to_ema_merged_file,
                             dtype = {"discriminated":str})
giant_table_EMA["PID"] = giant_table_EMA["PID"].apply(lambda x : "pid{0:0>3}".format(str(x)))
giant_table_EMA["date"] = pd.to_datetime(giant_table_EMA["date"])
# giant_table_EMA.set_index(["PID","date"], inplace=True)

# Get the table for Baseline

In [ ]:
giant_table_baseline = pd.read_csv(path_to_baseline_merged_file)
giant_table_baseline.rename(columns = {giant_table_baseline.columns[0]: "PID"},inplace = True)
giant_table_baseline["PID"] = giant_table_baseline["PID"].apply(lambda x : "pid{0:0>3}".format(x))
giant_table_baseline_pre = giant_table_baseline[giant_table_baseline["survey_type"] == "baseline2"]
giant_table_baseline_pre.set_index("PID", inplace = True)
giant_table_baseline_post = giant_table_baseline[giant_table_baseline["survey_type"] == "post"]
giant_table_baseline_post.set_index("PID", inplace = True)

In [ ]:
scales_interested = [
    "BDI_II",
    "CES_D",
    "PSS",
    "CHIPS",
    "UCLA_Loneliness",
    "ISEL",
    "BRS",
    "K2way_SSS",
    "MLE",
    "Sports",
    "SF_12",
    "MAAS",
    "GMM",
    "RSQ",
    "STAI",
    "ERQ",
    "BFI_Extraversion",
    "BFI_Agreeableness",
    "BFI_Conscientiousness",
    "BFI_Neuroticism",
    "BFI_Openness",
    "MSLQ",
]

In [ ]:
giant_table_baseline_pre_selected = giant_table_baseline_pre[scales_interested]
giant_table_baseline_pre_selected.columns = [x+"_PRE" for x in giant_table_baseline_pre_selected.columns]
giant_table_baseline_post_selected = giant_table_baseline_post[scales_interested]
giant_table_baseline_post_selected.columns = [x+"_POST" for x in giant_table_baseline_post_selected.columns]
giant_table_baseline_pre_post_selected = giant_table_baseline_pre_selected.merge(
                                            giant_table_baseline_post_selected,
                                            how = "outer",
                                            left_index = True,
                                            right_index = True)
giant_table_baseline_pre_post_selected.reset_index(inplace = True)

# Merge all tables

## Rebuild the date range

In [ ]:
#merge with largest, then remove the date
start_date = pd.to_datetime("2018-01-01")
end_date = pd.to_datetime("2018-06-30")

In [ ]:
date_range = pd.date_range(start_date, end_date)

In [ ]:
def rebuild_table_with_all_dates(df):
    giant_table_index_date = df.set_index("date")
    giant_table_grp_tmp = giant_table_index_date.groupby("PID")
    expanded_tablelist = []
    pid_count = 0
    for pid, giant_table_pid in giant_table_grp_tmp:
        giant_table_index = pd.DataFrame(index = date_range)
        giant_table_index = giant_table_index.merge(giant_table_pid, how = "outer",
                                                    left_index=True, right_index = True)
        giant_table_index["PID"] = pid
        expanded_tablelist.append(deepcopy(giant_table_index))
        pid_count += 1
    giant_table_rebuild = pd.concat(expanded_tablelist)
    giant_table_rebuild.reset_index(inplace = True)
    giant_table_rebuild.rename({"index":"date"}, axis = 1, inplace = True)
    return giant_table_rebuild

In [ ]:
giant_table_sensor_rebuild = rebuild_table_with_all_dates(giant_table_sensor)

In [ ]:
giant_table_sleep = rebuild_table_with_all_dates(giant_table_sleep)
giant_table_EMA = rebuild_table_with_all_dates(giant_table_EMA)

## Merge table by date and pid

In [ ]:
giant_table_sleep_EMA = giant_table_sleep.merge(giant_table_EMA,
                                                how = "outer",
                                                on = ["PID","date"])

In [ ]:
giant_table_sleep_EMA

In [ ]:
giant_table_sensor_sleep_EMA = giant_table_sensor_rebuild.merge(giant_table_sleep_EMA,
                                                how = "left",
                                                on = ["PID","date"])

In [ ]:
giant_table_sensor_sleep_EMA

In [ ]:
# giant_table_sleep_EMA_baseline = giant_table_sleep_EMA.merge(giant_table_baseline_pre_post_selected,
#                                                              how = "outer",
#                                                              on = ["PID"])

In [ ]:
giant_table_sensor_sleep_EMA_baseline = giant_table_sensor_sleep_EMA.merge(giant_table_baseline_pre_post_selected,
                                                             how = "left",
                                                             on = ["PID"])

In [ ]:
# Filter the date
giant_table_sensor_sleep_EMA_baseline = giant_table_sensor_sleep_EMA_baseline[
#     (giant_table_sensor_sleep_EMA_baseline["date"] >= pd.to_datetime("2018-1-21")) & 
    (giant_table_sensor_sleep_EMA_baseline["date"] <= pd.to_datetime("2018-6-15"))
]

In [ ]:
giant_table_sensor_sleep_EMA_baseline.to_csv(path_to_save_giant_table + "\giant_table_sensor_sleep_EMA_baseline.csv", index = False)

In [ ]:
giant_table_sensor_sleep_EMA_baseline.shape